Equação de posição:


In [ ]:
class Functions():
    
    def position_equation(self, initial_position:float, velocity:float, time:float) -> float:  #posição
        return initial_position + velocity*time

    def zero_array(self, length: int) -> list:
        
        result_array = []
        iterator = 0 
        
        while iterator < length:
            result_array.append(0)
            iterator += 1
        
        return result_array    
    
    def amax_like(array_to_find_max, axis ):
        return None
    
    def vstack_like():
        return None
    
    def root_function(self, number_to_root, )
    def sqrt_like(self, number_to_sqrt):
        return None

    

functions = Functions()

In [ ]:
class Simulation():
    def __init__(self, model_number, number_of_cars, track_length, time_step, total_time, correction_factor, initial_time):
        self.model_number = model_number
        self.number_of_cars = number_of_cars
        self.track_length = track_length
        self.track_length = track_length #fim_da_pista in meters
        self.time_steps = time_step #passos_tempo -> 
        self.total_time = total_time #tempo_total
        self.correction_factor = correction_factor #c
        self.initial_time = initial_time #t_inicial
        self.time_step = (self.total_time - self.initial_time)/self.time_steps

    def run_simulation(self, params):
        #need to create a vehicle
        v = np.ones(self.number_of_cars) * params['v_inicial']
        x_vec = np.linspace(0,self.track_length-self.track_length/6,self.number_of_cars)
        x_vec = x_vec[::-1] # inverter as posições 
        x_v_vec = np.concatenate(([x_vec], [v]), axis=0).reshape(1,-1)[0] # criar um vetor de velocidade
        ts = np.linspace(0,self.total_time,self.time_step)
        y_s = []
        y_s.append(x_v_vec)
        for i in range(1,len(ts)):
            y_s.append(runge_kutta_4(y_s[-1], x_v_dash, ts[i], ts[i]-ts[i-1], params))
        y_s = np.array(y_s)
        return y_s

In [ ]:
class Vehicle():
    def __init__(self, guided_by, desired_speed, initial_speed, time_between_cars, maximum_acceleration
    , maximum_slowdown, acceleration_exponent, minimum_interval, simulation):
        self.guided_by = guided_by
        self.desired_speed = desired_speed #v0
        self.initial_speed = initial_speed #v_inicial
        self.time_between_cars = time_between_cars #T
        self.maximum_acceleration = maximum_acceleration #a
        self.maximum_slowdown = maximum_slowdown #b
        self.acceleration_exponent = acceleration_exponent #delta
        self.minimum_interval = minimum_interval #s0 in meters
        self.track_length = simulation.track_length
        self.track_length = simulation.track_length #fim_da_pista in meters
        self.time_steps = simulation.time_step #passos_tempo -> 
        self.total_time = simulation.total_time #tempo_total
        self.correction_factor = simulation.correction_factor #c
        self.initial_time = simulation.initial_time #t_inicial
        self.time_step = simulation.time_step
        #self.number_of_cars = number_of_cars
    
    def s_asterisco(self, delta_v,v):
        zeros =  functions.zero_array(length = len(delta_v))
        return self.minimum_interval + np.amax(np.vstack((zeros, v*self.time_between_cars + v*delta_v/(2*functions.sqrt_like(self.maximum_acceleration * self.maximum_slowdown)))), axis = 0)
        #return s0 + np.amax(np.vstack((zeros,v*T + v*delta_v/(2*np.sqrt(a*b)))),axis=0) #Eq. 11.15
    
    def a_IDM(self, v,s,delta_v,params): #IDM
        return self.maximum_acceleration*(1-(v/self.desired_speed)**(self.acceleration_exponent) - (self.s_asterisco(v, delta_v)/self.s)**2) #Eq. 11.14
    
    def a_free_function(self, v):#page 196
        
        comp = (v <= self.desired_speed) # 1 or 0
        
        return (comp*(self.maximum_acceleration*(1-(v/self.desired_speed)**self.acceleration_exponent))
        +(1-comp)*(-self.maximum_slowdown*(1-(self.desired_speed/v)**
        (self.maximum_acceleration*self.acceleration_exponent/self.maximum_slowdown)))) #Eq. 11.22

    def a_IIDM(self, v,s,delta_v): #Improved IDM

        z = self.s_asterisco(v = v, delta_v = delta_v)/s
        v_comparation = (v <= self.desired_speed) # 1 or 0
        z_comparation = (z >= 1)
        a_free = self.a_free_function(v)
        dv_dt = np.zeros(v.shape)

        #Eq. 11.23 e 11.24
        dv_dt += v_comparation * z_comparation * (self.maximum_acceleration*(1-z**2))
        # Com v <= v0 e z < 1
        dv_dt += v_comparation * (1-z_comparation) * (a_free*(1-z**(2*self.maximum_acceleration/a_free)))
        # Com v > v0 e z >= 1
        dv_dt += (1-v_comparation) * z_comparation * (a_free+self.maximum_acceleration*(1-z**2))
        # Com v > v0 e z < 1
        dv_dt += (1-v_comparation) * z_comparation * (a_free)

        return dv_dt

    def a_CAH(self, s, v, vl, v_dash_l): #constant-acceleration heuristic Eq. 11.25
        ã_l = np.minimum(v_dash_l, self.maximum_acceleration)
        comp = ((vl*(v-vl))<=(-2*s*ã_l))
        return ((np.square(v)*ã_l/(np.square(vl) - 2*s*ã_l) * comp) + (ã_l - (np.square(v-vl)*((v-vl)>0)/(2*s)))*(1-comp))

    def a_ACC(self, s, v, vl, a_iidm): #ACC Eq. 11.26 

        v_dash_l = np.roll(a_iidm,1)
        a_cah = self.a_CAH(s, v, vl, v_dash_l)
        comp = (a_iidm >= a_cah) #1 or 0
        return ((a_iidm * comp) + ((1-self.correction_factor)*a_iidm +  self.correction_factor
        *(a_cah + self.maximum_slowdown*np.tanh((a_iidm-a_cah)/self.maximum_slowdown)))*(1-comp))

    def x_v_dash(self, x_v, t, simulation):
        x_v = x_v.reshape(2,-2) # derivada da posição e velocidade
        v = x_v[1,:] # obter velocidades
        x_vec = x_v[0,:] # obter posições
        # Pista circular, carro 0 segue o carro n-1, delta_v = v[i] - v[i-1]
        vl = np.roll(v,1)
        delta_v = v - vl # diferença entre as velocidades
        s = np.roll(x_vec,1) - x_vec
        s[0] += self.track_length
        if simulation.model_number == 1:
            dvdt = self.a_IDM(v,s,delta_v)
        else:
            dvdt = self.a_IIDM(v,s,delta_v)
        if simulation.model_number == 2:
            dvdt = self.a_ACC(s, v, vl, dvdt)
        x_v = np.concatenate((v,dvdt))
        return x_v

    def runge_kutta_4(self, x_v_vec_k, t_k, h, simulation): #Runge-Kutta 4th Order 
        k1=self.x_v_dash(x_v_vec_k,t_k, simulation)
        k2=self.x_v_dash(x_v_vec_k+0.5*h*k1,t_k+0.5*h, simulation)
        k3=self.x_v_dash(x_v_vec_k+0.5*h*k2,t_k+0.5*h, simulation)
        k4=self.x_v_dash(x_v_vec_k+h*k3,t_k+h, simulation)
        x_v_vec_k_next = x_v_vec_k + h/6.0 * (k1 + 2*k2 + 2*k3 + k4)
        #params['x_v_dash'].append(k1) n entendi essa parte
        return x_v_vec_k_next

#class IDM(Vehicle):
#class HDM(Vehicle):        
    

    params['passos_tempo'] = 1000 # numero de intervalo de tempo
    params['c'] = 0.99 # fator de correção
    params['t_inicial'] = 0.0 #tempo inicial
    


   
    params['Tr'] = 0.6 
    params['Vs'] = 0.1 
    params['sigma_r'] = 0.01 
    params['sigma_a']= 0.1  
    params['tau_til'] = 20.0 
    params['tau_a_til'] =  1.0 
    